In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#/kaggle/input/house-prices-advanced-regression-techniques
#import os
#if not os.path.exists("../input/train.csv"):
    #os.symlink('/kaggle/input/house-prices-advanced-regression-techniques/train.csv', "../input/train.csv")  
    #os.symlink("/kaggle/input/house-prices-advanced-regression-techniques/test.csv", "../input/test.csv") 

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
# Import helpful libraries
import catboost
import pandas as pd
#from sklearn.ensemble import RandomForestRegressor
#from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
#from xgboost import XGBRegressor , XGBRFRegressor
import numpy as np
# Load the data, and separate the target
iowa_file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
home_data = pd.read_csv(iowa_file_path)
#print(home_data.shape)
#home_data.SalePrice.quantile(.25)

In [3]:
#home_data.columns

In [4]:
def null_in_columns(df):
    for column in df:
        print(f'{column}: {df[column].isna().sum()}')

def print_unique_col_values(df):
       for column in df:
            if df[column].dtypes=='object':
                print(f'{column}: {df[column].unique()}') 
#print_unique_col_values(home_data)

In [5]:
'''
IQR outlier removal
'''
IQR_LotArea = home_data.LotArea.quantile(.75) - home_data.LotArea.quantile(.25)
LotArea_low = home_data.LotArea.quantile(.25) - 1.5*IQR_LotArea
LotArea_high = home_data.LotArea.quantile(.75) + 1.7*IQR_LotArea
print(LotArea_low,LotArea_high)
out=home_data[(home_data.LotArea>LotArea_high)|(home_data.LotArea<LotArea_low)]
out.shape

#outlier removed
h_data_clean = home_data[(home_data.LotArea<=LotArea_high)&(home_data.LotArea>=LotArea_low)]

1481.5 18483.1


In [6]:
LOT_mean = home_data.groupby(['Neighborhood'],group_keys=True)["LotFrontage"].mean()
#LOT_mean["Blueste"]



In [7]:
for i in h_data_clean.index:
    if  h_data_clean.MasVnrType[i]!=h_data_clean.MasVnrType[i]:
        if  h_data_clean.MasVnrArea[i]!=h_data_clean.MasVnrArea[i]:
            h_data_clean.MasVnrType[i] = "None"
            h_data_clean.MasVnrArea[i] = 0
        elif h_data_clean.MasVnrArea[i]==0: 
            h_data_clean.MasVnrType[i] = "None"
        else:
            h_data_clean.MasVnrType[i] = "BrkCmn"
    # MasVnrArea
    if  h_data_clean.MasVnrArea[i]!=h_data_clean.MasVnrArea[i]:
        if h_data_clean.MasVnrType[i]=="None":
            h_data_clean.MasVnrArea[i]= 0
        else:
            h_data_clean.MasVnrArea[i]= round(h_data_clean.MasVnrArea.mean())
    #MasVnrType
    if  h_data_clean.LotFrontage[i]!=h_data_clean.LotFrontage[i]:
        h_data_clean.LotFrontage[i] = LOT_mean[h_data_clean.Neighborhood[i]]
        
    if h_data_clean.BsmtQual[i]!=h_data_clean.BsmtQual[i]:
        h_data_clean.BsmtQual[i] = "NA"
        h_data_clean.BsmtCond[i] = "NA"
        h_data_clean.BsmtExposure[i] = "NA"
        h_data_clean.BsmtFinType1[i] = "NA"
        h_data_clean.BsmtFinType2[i] = "NA"
    if h_data_clean.BsmtExposure[i]!=h_data_clean.BsmtExposure[i]:
        if  h_data_clean.BsmtQual[i] == "NA":
            h_data_clean.BsmtExposure[i] ="NA"
            h_data_clean.BsmtFinType2[i] ="NA"
            
        else:
            h_data_clean.BsmtExposure[i] ="No"
            h_data_clean.BsmtFinType2[i] ="ALQ"
    if h_data_clean.BsmtFinType2[i]!=h_data_clean.BsmtFinType2[i]:
        if  h_data_clean.BsmtQual[i] == "NA":
            h_data_clean.BsmtExposure[i] ="NA"
            h_data_clean.BsmtFinType2[i] ="NA"
            
        else:
            h_data_clean.BsmtFinType2[i] ="ALQ"
    # MasVnrArea
    

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to b

In [8]:

h_data_clean.drop(["Alley","PoolQC","MiscFeature","Fence","GarageYrBlt"],axis ='columns' ,inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [9]:
print(round(h_data_clean.LotFrontage.mean()))
h_data_clean = h_data_clean.fillna({
    #"LotFrontage" : round(h_data_clean.LotFrontage.mean()),
    "MasVnrType" : "None",
    "MasVnrArea": 0 , #can be learned by ML it is related to the MasVnrType if not None
    #"BsmtQual": "NA" ,# can check to see if there is basement then estimate something
    'BsmtCond': "TA" ,
    #'BsmtExposure': "NA" ,
    #'BsmtFinType1': "NA" ,
    #'BsmtFinType2': "NA" ,
    'Electrical' : "Mix",
    'FireplaceQu': "NA",
    'GarageType': "NA",
    #'GarageYrBlt':round(h_data_clean.GarageYrBlt.mean()),
    'GarageFinish': "NA",
    'GarageQual': "NA",
    'GarageCond': "NA",
    #'PoolQC': "NA",
    #'Fence': "NA",
    #'Alley': "NA",
    #"MiscFeature": "NA"
})
h_data_clean["MSZoning"][0] = "RH"
h_data_clean.MSZoning.unique()

70


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


array(['RH', 'RL', 'RM', 'C (all)', 'FV'], dtype=object)

In [10]:
from sklearn.preprocessing import LabelEncoder
X = h_data_clean.drop(["SalePrice","Id"],axis='columns')
y = h_data_clean.SalePrice
cols_with_text = [col for col in X.columns #search for col with missing data
                     if pd.api.types.is_string_dtype(X[col].dtype)]
le_dic={}
for i in cols_with_text:
    le=LabelEncoder()
    #print(X[i].unique())
    X[i] = le.fit_transform(X[i])
    le_dic[i]=le

cols_with_missing = [col for col in X.columns #search for col with missing data
                     if X[col].isnull().any()]
print(cols_with_missing)
features = list(h_data_clean.keys()[:-1])

[]


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#scaler_x = StandardScaler()
scaler_x = MinMaxScaler()
X_scaled = scaler_x.fit_transform(X)
#y_scaled = scaler_y.fit_transform(y.reshape(1, -1))
X_scaled.shape

(1398, 74)

In [12]:
rf_model_on_full_data = catboost.CatBoostRegressor(depth=7,silent = True, iterations=500, learning_rate=0.1)
rf_model_on_full_data.fit(X_scaled,y)


In [13]:
test_data_path = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'

# read test data file using pandas
test_data = pd.read_csv(test_data_path)
col =X.columns
# create test_X which comes from test_data but includes only the columns you used for prediction.
# The list of columns is stored in a variable called features
#print(col)
test_X =test_data[col]
for i in test_X.index:
    if  test_X.MasVnrType[i]!=test_X.MasVnrType[i]:
        if  test_X.MasVnrArea[i]!=test_X.MasVnrArea[i]:
            test_X.MasVnrType[i] = "None"
            test_X.MasVnrArea[i] = 0
        elif test_X.MasVnrArea[i]==0: 
            test_X.MasVnrType[i] = "None"
        else:
            test_X.MasVnrType[i] = "BrkCmn"
    # MasVnrArea
    if  test_X.MasVnrArea[i]!=test_X.MasVnrArea[i]:
        if test_X.MasVnrType[i]=="None":
            test_X.MasVnrArea[i]= 0
        else:
            test_X.MasVnrArea[i]= round(test_X.MasVnrArea.mean())
    #MasVnrType
    if  test_X.LotFrontage[i]!=test_X.LotFrontage[i]:
        test_X.LotFrontage[i] = LOT_mean[test_X.Neighborhood[i]]
        
    if test_X.BsmtQual[i]!=test_X.BsmtQual[i]:
        test_X.BsmtQual[i] = "NA"
        test_X.BsmtCond[i] = "NA"
        test_X.BsmtExposure[i] = "NA"
        test_X.BsmtFinType1[i] = "NA"
        test_X.BsmtFinType2[i] = "NA"
    if test_X.BsmtExposure[i]!=test_X.BsmtExposure[i]:
        if  test_X.BsmtQual[i] == "NA":
            test_X.BsmtExposure[i] ="NA"
            test_X.BsmtFinType2[i] ="NA"
            
        else:
            test_X.BsmtExposure[i] ="No"
            test_X.BsmtFinType2[i] ="ALQ"
    if test_X.BsmtFinType2[i]!=test_X.BsmtFinType2[i]:
        if  test_X.BsmtQual[i] == "NA":
            test_X.BsmtExposure[i] ="NA"
            test_X.BsmtFinType2[i] ="NA"
            
        else:
            test_X.BsmtFinType2[i] ="ALQ"
    # MasVnrArea
#test_X.drop(["Alley","PoolQC","MiscFeature","Fence","GarageYrBlt"],axis ='columns' ,inplace=True)
test_X = test_X.fillna({
    "Functional" : "Typ",
    "MasVnrType" : "None",
    "MasVnrArea": 0 , #can be learned by ML it is related to the MasVnrType if not None
    "FireplaceQu": "NA" ,# can check to see if there is basement then estimate something
    'BsmtCond': "TA" ,
    'GarageCars': 0 ,
    'GarageArea': 0 ,
    'Exterior2nd': "Brk Cmn" ,
    'Electrical' : "Mix",
    'Utilities': "AllPub",
    'GarageType': "NA",
    'MSZoning': "RL",
    
    'Exterior1st':"BrkComm",
    'GarageFinish': "NA",
    'GarageQual': "NA",
    'GarageCond': "NA",
    'BsmtFullBath': 0,
    'BsmtHalfBath': 0,
    'BsmtFinSF1': 0,
    'BsmtFinSF2': 0,
    'BsmtUnfSF': 0,
    'TotalBsmtSF': 0,
    'KitchenQual': "TA",
    "SaleType": "WD"
})
test_X.RoofStyle.replace("Shed",'Mansard',inplace=True)
#test_X.MiscFeature.replace("Mansard",'None',inplace=True)
for i in cols_with_text:
    #print(i)
    test_X[i] = le_dic[i].transform(test_X[i])
X_scaled1 = scaler_x.transform(test_X)
#X_pca = pca.transform(X_scaled)
test_preds = rf_model_on_full_data.predict(X_scaled1)


#print_unique_col_values(test_X)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to b

In [14]:
#null_in_columns(test_X)
#test_X[["BsmtFinSF1","BsmtFinSF2","BsmtUnfSF","TotalBsmtSF"]][test_X.BsmtFinSF1.isnull()]
#h_data_clean.groupby(["RoofStyle"])["RoofStyle"].count()
#print_unique_col_values(h_data_clean)

In [15]:
#test_X.groupby(["RoofStyle"])["RoofStyle"].count()

In [16]:
#test_X.Functional.replace("Sal",'Maj2',inplace=True)
#print(test_X.RoofStyle.unique())
#"""
#test_X = test_X[test_X.Functional != 'Sev']


In [17]:
output = pd.DataFrame({'Id': test_data.Id,
                       'SalePrice': test_preds})
output.to_csv('submission.csv', index=False)